In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
#from tqdm.auto import tqdm
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 22/22 [00:10<00:00,  2.00it/s]


In [4]:
%matplotlib qt
import skimage.io as io
import xarray as xr
from collections import OrderedDict
import dask
from tqdm.dask import TqdmCallback
from sklearn.metrics.pairwise import euclidean_distances
from matplotlib import cm

In [5]:
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_hdf(h5_filename, wd=filedir)

Loading data from hdf file...
Loading most recent images (raw_images)...done!
Loading reciprocal positions...done!
Instantiating ImageMap...done!
XRD Map loaded!


In [10]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
test.load_phase('AMCSD\\Silicon_0011243.cif', filedir=cif_dir, phase_name="silicon")
silicon = test.phases['silicon']
test.load_phase('AMCSD\\Corundum_0009327.cif', filedir=cif_dir, phase_name="corundum")
corundum = test.phases['corundum']
test.load_phase('AMCSD\\Zincite_0011555.cif', filedir=cif_dir, phase_name="zincite")
zincite = test.phases['zincite']


In [11]:
all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['zincite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

ax.scatter(*np.asarray(all_qs).T, c='k', s=all_fs/20)
ax.scatter(0, 0, 0, c='k', s=10)

'''# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')'''

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [18]:
silicon.Q(1, 1, 1)

array([1.15697522, 1.15697522, 1.15697522])

In [76]:
e11, e12, e13, e22, e23, e33 = 1e-6, 2e-6, -3e-6, 1e-6, -3e-6, 2e-6
w1, w2, w3 = 3e-5, 5e-5, 1e-4

eij = np.asarray([[e11, e12, e13], [e12, e22, e23], [e13, e23, e33]])
wij = np.asarray([[0, -w3, w2], [w3, 0, -w1], [-w2, w1, 0]])
I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

duij = eij + I

In [77]:
phi_x, phi_y, phi_z = 30, 40, 60

phi_x, phi_y, phi_z = np.radians([phi_x, phi_y, phi_z])

Rx = np.asarray([[1, 0, 0],
                 [0, np.cos(phi_x), -np.sin(phi_x)],
                 [0, np.sin(phi_x), np.cos(phi_x)]])

Ry = np.asarray([[np.cos(phi_y), 0, np.sin(phi_y)],
                 [0, 1, 0],
                 [-np.sin(phi_y), 0, np.cos(phi_y)]])

Rz = np.asarray([[np.cos(phi_z), -np.sin(phi_z), 0],
                 [np.sin(phi_z), np.cos(phi_z), 0],
                 [0, 0, 1]])

rot = Rx @ Ry @ Rz

Fij =  duij @ rot

In [94]:
Fij

array([[ 0.38302396, -0.6634165 ,  0.6427855 ],
       [ 0.91069812,  0.15467413, -0.38302331],
       [ 0.15467393,  0.7320937 ,  0.6634145 ]])

In [79]:
rot

array([[ 0.38302222, -0.66341395,  0.64278761],
       [ 0.9106969 ,  0.1546775 , -0.38302222],
       [ 0.1546775 ,  0.73209071,  0.66341395]])

In [80]:
from scipy.linalg import polar

In [107]:
p, u = linalg.polar(Fij, side='left')
print(u)
print(p)

[[ 1.000001  0.000002 -0.000003]
 [ 0.000002  1.000001 -0.000003]
 [-0.000003 -0.000003  1.000002]]
[[ 0.38302222 -0.66341395  0.64278761]
 [ 0.9106969   0.1546775  -0.38302222]
 [ 0.1546775   0.73209071  0.66341395]]


In [110]:
print(u - duij)

[[ 0.  0.  0.]
 [ 0.  0. -0.]
 [ 0.  0.  0.]]


In [111]:
u

array([[ 1.000001,  0.000002, -0.000003],
       [ 0.000002,  1.000001, -0.000003],
       [-0.000003, -0.000003,  1.000002]])

In [123]:
rec_duij = u - I
rec_eij = np.round(0.5 * (rec_duij.T + rec_duij), 10)
rec_wij = np.round(0.5 * (rec_duij - rec_duij.T), 10)

In [124]:
rec_e11 = rec_eij[0, 0]
rec_e12 = rec_eij[0, 1]
rec_e13 = rec_eij[0, 2]
rec_e22 = rec_eij[1, 1]
rec_e23 = rec_eij[1, 2]
rec_e33 = rec_eij[2, 2]

In [128]:
print(f'ε₁₁ is {rec_e11 * 1e6} µε.')
print(f'ε₂₂ is {rec_e22 * 1e6} µε.')
print(f'ε₃₃ is {rec_e33 * 1e6} µε.')
print(f'ε₁₂ is {rec_e12 * 1e6} µε.')
print(f'ε₁₃ is {rec_e13 * 1e6} µε.')
print(f'ε₂₃ is {rec_e23 * 1e6} µε.')

ε₁₁ is 1.0 µε.
ε₂₂ is 1.0 µε.
ε₃₃ is 2.0 µε.
ε₁₂ is 2.0 µε.
ε₁₃ is -3.0 µε.
ε₂₃ is -3.0 µε.
